In [43]:
import requests
import urllib
import pandas as pd
from glob import glob
import json
import time
import tqdm

In [2]:
cmc_apikey = "69328ced-5543-43f5-80b0-7171f6f7c70d"
coinapi_apikey = "3DDA805E-5521-415B-9C67-0C08B3FECA5F"

In [3]:
d = {"name": "yasmine", "age": 30}
d.get('job', [])

[]

In [4]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories"
R = requests.request("get", url, headers={
    "X-CMC_PRO_API_KEY": cmc_apikey #"sitepyheadersparameter"-<to provide APIkey
})
print("Request completed")

content = json.loads(R.content.decode("utf-8"))
cat_df = pd.DataFrame(content.get('data', []))
cat_df.to_excel("cat_df.xlsx", index=False)
cat_df

Request completed


,id,name,title,description,num_tokens,avg_price_change,market_cap,market_cap_change,volume,volume_change,last_updated
0,6363a6c9cd197958bb543bf0,Soccer,Soccer,Soccer,50,-1.629632,1.187571e+08,-1.408910,1.975424e+07,1091.439431,2022-11-03T11:36:20.885Z
1,635634b93e16136f3bb2ee9e,Aptos Ecosystem,Aptos Ecosystem,Aptos Ecosystem,6,3.896835,4.109273e+08,-0.075800,6.534546e+07,-13.181140,2022-10-24T06:46:17.004Z
2,63330401558c5024ac5dade9,Internet Computer Ecosystem,Internet Computer Ecosystem,Internet Computer Ecosystem,3,-7.006667,1.111859e+09,-0.005800,2.541989e+07,116.674650,2022-09-27T14:09:39.579Z
3,63258b2a694d2a40b4040007,Farming as a Service (FaaS),Farming as a Service (FaaS),Farming as a Service (FaaS),9,-1.646169,0.000000e+00,0.000000,5.675507e+04,60.913322,2022-09-17T08:55:03.546Z
4,62fe5eb097edc244f4716518,Doge Chain Ecosystem,Doge Chain Ecosystem,Doge Chain,11,-1.625414,0.000000e+00,0.000000,9.670440e+05,-4.845118,2022-08-22T04:40:45.482Z
...,...,...,...,...,...,...,...,...,...,...,...
165,60308028d2088f200c58a005,BNB Chain Ecosystem,Top BNB Chain Tokens by Market Capitalization,BNB Chain dual-chain architecture will empower...,2000,722167.746078,1.839026e+11,0.076784,2.272267e+10,29331.041232,2022-02-28T09:24:12.374Z
166,60291fa0db1be76c46298e83,NFTs & Collectibles,Top NFTs & Collectibles Tokens by Market Capit...,A NFT (non-fungible token) is a special crypto...,609,35.073332,9.436390e+09,36.231373,7.054707e+08,1609.354569,2021-05-04T10:36:15.891Z
167,601cf8d2d8fd860e4ea5d96f,Polkadot Ecosystem,Top Polkadot Tokens by Market Capitalization,Polkadot is an open-source sharding multichain...,118,-1.156146,9.864749e+09,-0.999053,4.769297e+08,32.902139,2021-03-23T08:38:28.669Z
168,5fb62da404d1dd4c73744883,Storage,Top Storage Tokens by Market Capitalization,In blockchain-based decentralized cloud storag...,40,1.530715,2.433065e+09,1.312073,1.282421e+08,52.307497,2021-03-23T08:38:28.660Z


In [5]:
category_id = '635634b93e16136f3bb2ee9e'
def get_coin_data(category_id):
    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/category?id={category_id}&aux=volume_7d,volume_30d"
    R = requests.request("get", url, headers={
        "X-CMC_PRO_API_KEY": cmc_apikey #"sitepyheadersparameter"-<to provide APIkey
    })
#     print("Request completed")

    content = json.loads(R.content.decode("utf-8"))
    content = content.get('data', {}) #Return empty dict if 'data' is not found in keys
    content = content.get('coins', []) #Return empty dict if 'coins' is not found in keys
    cat_data = pd.DataFrame(content)
    cat_data['category_id'] = category_id
    
    return cat_data

In [6]:
coin_df = []
for x in tqdm.tqdm(cat_df['id'].unique()):
    coin_df.append(get_coin_data(x))
    time.sleep(2)

coin_df = pd.concat(coin_df)

  4%|███▍                                                                              | 7/170 [00:17<06:52,  2.53s/it]

KeyboardInterrupt



In [ ]:
#coin_df.to_csv
coin_df.to_excel("coin_df.xlsx", index=False)

In [ ]:
cat_df.sample()

In [ ]:
coin_df.sample()

In [ ]:
cat_coin_df = cat_df.merge(right=coin_df, left_on="id", right_on="category_id",suffixes=('_category','_coin'))
cat_coin_df.sample()

In [ ]:
with open('cat_coin_df.json', 'w', encoding='utf8') as f:
    json.dump(cat_coin_df.to_dict('r'), f)

In [9]:
with open('cat_coin_df.json', 'r', encoding='utf8') as f:
    cat_coin_df=pd.DataFrame(json.load(f))
#

In [10]:
cat_coin_df

,id_category,name_category,title,description,num_tokens,avg_price_change,market_cap,market_cap_change,volume,volume_change,...,platform,is_active,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated_coin,quote,category_id
0,6363a6c9cd197958bb543bf0,Soccer,Soccer,Soccer,50,-1.688168,1.222782e+08,-0.979357,1.735982e+07,91.385333,...,"{'id': 4066, 'name': 'Chiliz', 'symbol': 'CHZ'...",1,516.0,0,3.840985e+06,2.110991e+07,NaN,2022-12-28T13:19:00.000Z,"{'USD': {'price': 5.495962781423792, 'volume_2...",6363a6c9cd197958bb543bf0
1,6363a6c9cd197958bb543bf0,Soccer,Soccer,Soccer,50,-1.688168,1.222782e+08,-0.979357,1.735982e+07,91.385333,...,"{'id': 4066, 'name': 'Chiliz', 'symbol': 'CHZ'...",1,537.0,0,5.403968e+06,1.925329e+07,NaN,2022-12-28T13:19:00.000Z,"{'USD': {'price': 3.5628064975277587, 'volume_...",6363a6c9cd197958bb543bf0
2,6363a6c9cd197958bb543bf0,Soccer,Soccer,Soccer,50,-1.688168,1.222782e+08,-0.979357,1.735982e+07,91.385333,...,None,1,569.0,0,NaN,NaN,NaN,2022-12-28T13:19:00.000Z,"{'USD': {'price': 4.021028716235134, 'volume_2...",6363a6c9cd197958bb543bf0
3,6363a6c9cd197958bb543bf0,Soccer,Soccer,Soccer,50,-1.688168,1.222782e+08,-0.979357,1.735982e+07,91.385333,...,"{'id': 4066, 'name': 'Chiliz', 'symbol': 'CHZ'...",1,687.0,0,4.094932e+06,1.014840e+07,NaN,2022-12-28T13:19:00.000Z,"{'USD': {'price': 2.478283993873156, 'volume_2...",6363a6c9cd197958bb543bf0
4,6363a6c9cd197958bb543bf0,Soccer,Soccer,Soccer,50,-1.688168,1.222782e+08,-0.979357,1.735982e+07,91.385333,...,"{'id': 4066, 'name': 'Chiliz', 'symbol': 'CHZ'...",1,708.0,0,4.162368e+06,9.541826e+06,NaN,2022-12-28T13:19:00.000Z,"{'USD': {'price': 2.2924032739733513, 'volume_...",6363a6c9cd197958bb543bf0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6856,5fb62883c9ddcc213ed13308,DeFi,Top DeFi Tokens by Market Capitalization,DeFi (decentralized finance) is the creation o...,569,-1.087572,3.598847e+10,-1.025701,1.851599e+09,135.017115,...,None,1,7794.0,0,NaN,NaN,NaN,2022-12-28T13:22:00.000Z,"{'USD': {'price': 0.9995800560289093, 'volume_...",5fb62883c9ddcc213ed13308
6857,5fb62883c9ddcc213ed13308,DeFi,Top DeFi Tokens by Market Capitalization,DeFi (decentralized finance) is the creation o...,569,-1.087572,3.598847e+10,-1.025701,1.851599e+09,135.017115,...,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",1,7795.0,0,NaN,NaN,NaN,2022-12-28T13:22:00.000Z,"{'USD': {'price': 0.9997889577386971, 'volume_...",5fb62883c9ddcc213ed13308
6858,5fb62883c9ddcc213ed13308,DeFi,Top DeFi Tokens by Market Capitalization,DeFi (decentralized finance) is the creation o...,569,-1.087572,3.598847e+10,-1.025701,1.851599e+09,135.017115,...,"{'id': 1376, 'name': 'Neo', 'symbol': 'NEO', '...",1,6769.0,0,3.711739e+09,8.737838e+07,NaN,2022-12-28T13:22:00.000Z,"{'USD': {'price': 0.023541093569672142, 'volum...",5fb62883c9ddcc213ed13308
6859,5fb62883c9ddcc213ed13308,DeFi,Top DeFi Tokens by Market Capitalization,DeFi (decentralized finance) is the creation o...,569,-1.087572,3.598847e+10,-1.025701,1.851599e+09,135.017115,...,None,0,NaN,0,6.866828e+07,4.436245e+06,NaN,2021-06-14T08:50:49.000Z,"{'USD': {'price': 0.06460399669798, 'volume_24...",5fb62883c9ddcc213ed13308


In [11]:
cat_coin_df[cat_coin_df['name_coin'] == "XRP"].T

,2039,2128,2449
id_category,605e2e38d41eae1066535f7f,605e2ce9d41eae1066535f7c,605bc602972c9053da7021a9
name_category,Pantera Capital Portfolio,a16z Portfolio,Arrington XRP Capital Portfolio
title,Pantera Capital Portfolio,a16z Portfolio,Arrington XRP Capital Portfolio
description,Pantera Capital Portfolio,a16z Portfolio,Arrington XRP Capital Portfolio
num_tokens,53,19,37
avg_price_change,-3.775613,-2.802133,-3.182584
market_cap,500852998129.329956,493405238724.459961,11282895441.940001
market_cap_change,-3.5552,-2.802889,-2.920563
volume,22914577591.17622,22299644502.85379,672007448.368965
volume_change,17.648112,22.167374,20.08638


In [18]:
# symbol = "BTC"
def get_time_series(symbol):
#     symbol = symbol.upper()
    url = f"https://rest.coinapi.io/v1/exchangerate/{symbol}/USD/history?period_id=1HRS&limit=200&time_start=2022-12-23T00:00:00&time_end=2022-12-30T00:00:00"
    R = requests.request("get", url, headers={
        "X-CoinAPI-Key": coinapi_apikey #"sitepyheadersparameter"-<to provide APIkey
    })
#     print("Request completed: ", R.status_code)
    if R.status_code != 200:
        return R.content
    
    
    content = json.loads(R.content.decode("utf-8"))
    ts_df = pd.DataFrame(content)
    ts_df['symbol'] = symbol
    return ts_df

In [20]:
all_paths = glob("timeseries/*.json")
for x in tqdm.tqdm(cat_coin_df['symbol'].unique()):
    p = f'timeseries/{x}.json'
    if p in all_paths:
        continue
    
    tsdf = get_time_series(x)
    with open(p, 'w', encoding='utf8') as f:
        json.dump(tsdf.to_dict('records'), f)
        
    time.sleep(2)
#use json instead of csv for dict problems a dict-->to a string

  4%|███▏                                                                         | 126/3068 [05:54<2:17:48,  2.81s/it]


AttributeError: 'bytes' object has no attribute 'to_dict'

In [52]:
# symbol = "BTC"
def get_metadata(symbol):
    symbol = symbol.upper()
#     symbol = urllib.parse.unquote(symbol)
    symbol = symbol.replace(' ', '-')
    url = f"https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?symbol={symbol}"
    R = requests.request("get", url, headers={
        "X-CMC_PRO_API_KEY": cmc_apikey #"sitepyheadersparameter"-<to provide APIkey
    })
    if R.status_code != 200:
        print("Request completed: ", R.status_code)
        return R.content
    
    
    content = json.loads(R.content.decode("utf-8"))
    meta_df = pd.DataFrame(content['data'][symbol])
    return meta_df

In [66]:
all_paths = glob("metadata/*.json")
for x in tqdm.tqdm(cat_coin_df['symbol'].unique()):
    p = f'metadata\\{x}.json'
    if p in all_paths:
        continue
    tsdf = get_metadata(x)   
    if type(tsdf) != bytes:
        with open(p, 'w', encoding='utf8') as f:
            json.dump(tsdf.to_dict('records'), f)
        time.sleep(2)
    else:
        continue
#use json instead of csv for dict problems a dict-->to a string
#we skipped eg'POO DOGE' for its space charcteristic problem
#68/3068
#247/3068
#

  2%|█▊                                                                             | 68/3068 [00:00<00:12, 239.99it/s]

Request completed:  400


  8%|██████▎                                                                       | 247/3068 [00:00<00:05, 502.78it/s]

Request completed:  400


 39%|██████████████████████████████                                               | 1197/3068 [00:03<00:04, 374.32it/s]

...

Request completed:  400


 54%|█████████████████████████████████████████▊                                    | 1645/3068 [09:49<34:26,  1.45s/it]

Request completed:  400


 59%|█████████████████████████████████████████▊                             | 1807/3068 [51:34:15<35:59:17, 102.74s/it]


ConnectionError: HTTPSConnectionPool(host='pro-api.coinmarketcap.com', port=443): Max retries exceeded with url: /v2/cryptocurrency/info?symbol=MELOS (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002C7CDAD2D60>: Failed to establish a new connection: [WinError 10065] Une opération a été tentée sur un hôte impossible à atteindre'))

In [ ]:
M = get_metadata("ETH")
M = M.set_index('symbol')

In [ ]:
(M['urls'].values[0])
#This must become its own dataframe

In [ ]:
import pandas as pd 
list = ['urls']  
df = pd.DataFrame(list, columns =[]) 

In [65]:
symbol = x.upper()
#     symbol = urllib.parse.unquote(symbol)
symbol = symbol.replace(' ', '+')
url = f"https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?symbol={symbol}"
R = requests.request("get", url, headers={
    "X-CMC_PRO_API_KEY": cmc_apikey #"sitepyheadersparameter"-<to provide APIkey
})

content = json.loads(R.content.decode("utf-8"))
meta_df = pd.DataFrame(content['data'][symbol])
meta_df

,id,name,symbol,category,description,slug,logo,subreddit,notice,tags,...,urls,platform,date_added,twitter_username,is_hidden,date_launched,contract_address,self_reported_circulating_supply,self_reported_tags,self_reported_market_cap
0,8845,Lepricon,L3P,token,Lepricon (L3P) is a cryptocurrency and operate...,lepricon-l3p,https://s2.coinmarketcap.com/static/img/coins/...,,L3P has been migrated to [LPR](https://coinmar...,"[duckstarter, poolz-finance-portfolio]",...,"{'website': ['https://lepricon.io/'], 'twitter...","{'id': '1027', 'name': 'Ethereum', 'slug': 'et...",2021-03-17T00:00:00.000Z,lepriconio,0,None,[{'contract_address': '0xdef1da03061ddd2a5ef6c...,None,None,None


In [50]:
x

'POO DOGE'